In [2]:
!pip install tensorflow

   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/390.3 MB ? eta -:--:--
   ---------------------------------------- 1.6/390.3 MB 7.0 MB/s eta 0:00:56
   ---------------------------------------- 3.1/390.3 MB 6.1 MB/s eta 0:01:03
    --------------------------------------- 5.8/390.3 MB 7.8 MB/s eta 0:00:50
    --------------------------------------- 8.1/390.3 MB 8.8 MB/s eta 0:00:44
   - -------------------------------------- 10.5/390.3 MB 9.2 MB/s eta 0:00:42
   - -------------------------------------- 13.1/390.3 MB 9.7 MB/s eta 0:00:39
   - -------------------------------------- 14.7/390.3 MB 9.9 MB/s eta 0:00:38
   - -------------------------------------- 18.1/390.3 MB 10.2 MB/s eta 0:00:37
   -- ------------------------------------- 20.2/390.3 MB 10.4 MB/s eta 0:00:36
   -- ------------------------------------- 23.1/390.3 MB 10.5 MB/s eta 0:00:35
   -- ------------------------------------- 25.4/390.3 MB 10.6 MB/s e

In [22]:
!pip install opencv-python


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
    --------------------------------------- 0.5/38.8 MB 4.2 MB/s eta 0:00:10
   - -------------------------------------- 1.8/38.8 MB 6.3 MB/s eta 0:00:06
   --- ------------------------------------ 3.1/38.8 MB 6.8 MB/s eta 0:00:06
   ---- ----------------------------------- 4.5/38.8 MB 6.0 MB/s eta 0:00:06
   ------ --------------------------------- 6.3/38.8 MB 6.8 MB/s eta 0:00:05
   -------- ------------------------------- 8.7/38.8 MB 7.7 MB/s eta 0:00:04
   ----------- ---------------------------- 11.3/38.8 MB 8.4 MB/s eta 0:00:04
   -------------- ------------------------- 13.6/38.8 MB 8.8 MB/s eta 0:00:03
   --------------- ------------------------ 15.5/38.8 MB 8.7 MB/s eta 0:00:03
   ------------------ --------------------- 17.8/38.8 MB 9.0 MB/s eta 0:00:03
   -------------------- ------------------- 19.7/38.8 MB 9.0 MB/s eta 0:00:03
   ---------------------- ----------------- 21.5/38.8 MB 9.1 MB/s eta 0:00:02


In [25]:
import numpy as np
import matplotlib.pyplot as plt


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [27]:
import os

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU is available")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("GPU is not available")

In [ ]:
tf.config.list_physical_devices('GPU')

In [28]:
import cv2
import imghdr

C:\Users\Shubh\AppData\Local\Temp\ipykernel_14708\4232469594.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [ ]:

data_dir = 'data' 
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))

### Load Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

### Sccale Data

In [ ]:
data = data.map(lambda x,y: (x/255, y))


In [ ]:
data.as_numpy_iterator().next()

### Split Data

In [ ]:

train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train_size


In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

### DL Model

In [4]:
model = Sequential()

In [7]:
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model.compile('adam', loss = tf.losses.BinaryCrossentropy(), metrics = ['accuracy']) 

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape          ┃      Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)               │ (None, 254, 254, 16)  │          448 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ conv2d_1 (Conv2D)             │ (None, 252, 252, 16)  │        2,320 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 126, 126, 16)  │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ conv2d_2 (Conv2D)             │ (None, 124, 124, 32)  │        4,640 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ max_pooling2d_1               │ (None, 62, 62, 32)    │            0 │
│ (MaxPooling2D)                │                       │              │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ conv2d_3 (Conv2D)             │ (None, 60, 60, 16)    │        4,624 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ max_pooling2d_2               │ (None, 30, 30, 16)    │            0 │
│ (MaxPooling2D)                │                       │              │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ flatten (Flatten)             │ (None, 14400)         │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ dense (Dense)                 │ (None, 256)           │    3,686,656 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ dense_1 (Dense)               │ (None, 1)             │          257 │
└───────────────────────────────┴───────────────────────┴──────────────┘

 Total params: 3,698,945 (14.11 MB)

 Trainable params: 3,698,945 (14.11 MB)

 Non-trainable params: 0 (0.00 B)

### Train

In [11]:
logdir = 'logs'

In [12]:
tensorboard_callbacl = tf.keras.callbacks.TensorBoard(log_dir = logdir)

In [13]:
hist = model.fit(train, epochs = 20, validation_data = val, callbacks = [tensorboard_callback])

NameError: name 'train' is not defined

### Evaluation

In [14]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [15]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [16]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

NameError: name 'test' is not defined

In [18]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy: {acc.result().numpy()}')

Precision: 0.0, Recall:0.0, Accuracy: 0.0


### Test

In [ ]:
img = cv2.imread('')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256, 256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [26]:
yhat = model.predict(np.expand_dims(resize/255, 0))

NameError: name 'resize' is not defined

In [ ]:
yhat